#### **Import Statements:**

In [ ]:
from collections import namedtuple
from enum import Enum
from PIL import Image, ImageOps
from skimage.viewer import ImageViewer
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor

import glob
import itertools
import matplotlib.pyplot as plt
import numpy as np
import os
import skimage
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data
import torchvision.transforms as transforms

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: Viewer requires Qt
  after removing the cwd from sys.path.


#### **Data Loading:**

In [ ]:
# USER CUSTOMIZATIONS - dataloader_path of images

# dir_input - stores original images
# dir_output - stores resized original images
# dir_augmented - stores dir_output after augmentations
dir_input = "/content/drive/MyDrive/APS360 Project Folder/Project/3. Implementation/Trial 3/1-original/"
dir_output = "/content/drive/My Drive/APS360 Project Folder/Project/3. Implementation/Trial 3/2-standarized/"
dir_augmented = "/content/drive/MyDrive/APS360 Project Folder/Project/3. Implementation/Trial 3/3-augmented/"

dataloader_path = 'dataset-v3/3B-bldg-Mirror-Rotate-2K'
dataloader_pkl_path = "/content/drive/MyDrive/APS360 Project Folder/Project/3. Implementation/Trial 3/3B-bldg-Mirror-Rotate-2K-unique.pkl"


# uncomment if using Google Colab to train
from google.colab import drive
drive.mount('/content/drive')

root = '/content/drive/My Drive/APS360 Project Folder/Project/3. Implementation/Trial 3/'

Mounted at /content/drive


In [ ]:
class DLabel(Enum):
    FAKE = 0
    REAL = 1

# (x1, y1) - top left; (x2, y2) - bottom right
RegionRect = namedtuple('RegionRect', ['x1', 'y1', 'x2', 'y2'])

New dataset used - https://www.kaggle.com/nitishabharathi/scene-classification

In [ ]:
# NOTE: only need to be called oce to save augmented images to folder
# image_size_standarization(128, dir_input, dir_output)

In [ ]:
augmentations = {
    "Greyscale": [True, False],
    "Mirror": ["ymirror", None],
    "Rotation": [180, None],
}

# augmentations = {
#     "Mirror": ["ymirror", None],
#     "Rotation": [180, None],
# }

# augmentations = {
#     "Greyscale": [True, False],
#     "Mirror": ["ymirror", None],
# }

# image_augmentation(augmentations, dir_output, dir_augmented)

In [ ]:
# Custom Dataset Class

class InpaintingDataset(Dataset):
    def __init__(self, image_dir, transform):
        self.image_dir = image_dir
        self.transform = transform
        self.image_names = os.listdir(image_dir)
        self.image_names.sort()

    def __len__(self):
        return len(self.image_names)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        image_names = os.path.join(self.image_dir, self.image_names[idx])
        image = Image.open(image_names)

        if self.transform:
            image = self.transform(image)
        return image

# Create InpaintingDataset instance
dataset = None
# dataset = InpaintingDataset(dataloader_path, None)

In [ ]:
def data_loader(bs=2, path=dataloader_path, t_ind=0.75, v_ind=0.15, o_num=2, dataset=dataset):
    """
    Returns a tuple with 4 elements that is ordered as following:
    ['training', 'validation', 'testing', 'overfitting']

    Each element contains loaded datasets
    """
    start_time = time.time()
    loader = []
    t_end = int(len(dataset) * t_ind)
    v_end = int(len(dataset) * (t_ind + v_ind))

    np.random.seed(1500)  # Fixed numpy random seed for reproducible shuffling
    np.random.shuffle(dataset)
    data_split = [
        dataset[:t_end]
        , dataset[t_end:v_end]
        , dataset[v_end:]
        , dataset[:o_num]
    ]
    print(len(dataset))
    load_order = ['training', 'validation', 'testing', 'overfitting']
    order = 0
    for split in data_split:
        print(f"Loading images for {load_order[order]}. Images to be loaded: {len(data_split[order])}")
        loader.append(DataLoader(split, batch_size=bs, shuffle = True))
        order += 1

    end_time = time.time()
    elapsed_time = end_time - start_time
    print("Total Time Elapsed: {:.2f} seconds".format(elapsed_time))

    return tuple(loader)

In [ ]:
import pickle

# dataset_combined = create_combined_list(mask_size=24, image_size=128)
#
# with open(dataloader_pkl_path, 'wb') as output:
#     pickle.dump(dataset_combined, output, 4)

with open(dataloader_pkl_path, 'rb') as input:
    dataset_combined = pickle.load(input)

#### **Generator:**

In [ ]:
# # ======================== Architecture 1: Encoder + Dilation========================
# class Generator(nn.Module):
#     def __init__(self):
#         super(Generator, self).__init__()
#         self.encoder = nn.Sequential(
#             nn.Conv2d(in_channels=4,  out_channels=32, kernel_size=2, stride=1, padding=1),
#             nn.BatchNorm2d(32),
#             nn.ReLU(True),
#             nn.Conv2d(in_channels=32,  out_channels=64, kernel_size=2, stride=1, padding=1),
#             nn.BatchNorm2d(64),
#             nn.ReLU(True),
#             nn.Conv2d(in_channels=64,  out_channels=128, kernel_size=2, stride=2, padding=1),
#             nn.BatchNorm2d(128),
#             nn.ReLU(True),
#             nn.Conv2d(in_channels=128,  out_channels=256, kernel_size=2, stride=2, padding=1),
#             nn.BatchNorm2d(256),
#             nn.ReLU(True),
#         )

#         self.dilation = nn.Sequential(
#             nn.Conv2d(in_channels=256,  out_channels=256, kernel_size=2, stride=1, padding=1, dilation=4),
#             nn.BatchNorm2d(256),
#             nn.ReLU(True),
#             nn.Conv2d(in_channels=256,  out_channels=256, kernel_size=2, stride=1, padding=0, dilation=8),
#             nn.BatchNorm2d(256),
#             nn.ReLU(True),
#         )

#         # decoder here is used for decreasing number of channels, not for increasing dimension of the image
#         self.decoder = nn.Sequential(
#             nn.Conv2d(in_channels=256,  out_channels=128, kernel_size=1, stride=1, padding=0),
#             nn.BatchNorm2d(128),
#             nn.ReLU(True),
#             nn.Conv2d(in_channels=128,  out_channels=64, kernel_size=1, stride=1, padding=0),
#             nn.BatchNorm2d(64),
#             nn.ReLU(True),
#             nn.Conv2d(in_channels=64,  out_channels=32, kernel_size=1, stride=1, padding=0),
#             nn.BatchNorm2d(32),
#             nn.ReLU(True),
#             nn.Conv2d(in_channels=32,  out_channels=3, kernel_size=1, stride=1, padding=0),
#             nn.Tanh()
#         )

#     def forward(self, x):
#         x = self.encoder(x)
#         x = self.dilation(x)
#         x = self.decoder(x)
#         return x

# ======================== Architecture 2: Autoencoder + Dilation ========================
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(in_channels=4,  out_channels=32, kernel_size=2, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(True),
            nn.Conv2d(in_channels=32,  out_channels=64, kernel_size=2, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            nn.Conv2d(in_channels=64,  out_channels=128, kernel_size=2, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            nn.Conv2d(in_channels=128,  out_channels=256, kernel_size=2, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
        )

        self.dilation = nn.Sequential(
            nn.Conv2d(in_channels=256,  out_channels=256, kernel_size=2, stride=1, padding=1, dilation=4),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            nn.Conv2d(in_channels=256,  out_channels=256, kernel_size=2, stride=1, padding=1, dilation=8),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            nn.Conv2d(in_channels=256,  out_channels=256, kernel_size=2, stride=1, padding=1, dilation=16),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
        )

        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(in_channels=256,  out_channels=128, kernel_size=1, stride=1, padding=0, output_padding=0),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            nn.ConvTranspose2d(in_channels=128,  out_channels=64, kernel_size=1, stride=1, padding=0, output_padding=0),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            nn.ConvTranspose2d(in_channels=64,  out_channels=32, kernel_size=1, stride=1, padding=0, output_padding=0),
            nn.BatchNorm2d(32),
            nn.ReLU(True),
            nn.ConvTranspose2d(in_channels=32,  out_channels=3, kernel_size=2, stride=2, padding=0, output_padding=0),
            nn.Tanh()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.dilation(x)
        x = self.decoder(x)
        return x

#### **Discriminator:**

In [ ]:
# 2 convolutional layers, this perfomed better than the 5 layered architecture
# during the overfitting test
class Discriminator(nn.Module):
  def __init__(self):
    super(Discriminator, self).__init__()
    self.main = nn.Sequential (
        nn.Conv2d(in_channels=3,  out_channels=8, kernel_size=5, stride=2, padding=1),
        nn.BatchNorm2d(8),
        nn.LeakyReLU(),
        nn.Conv2d(in_channels=8, out_channels=16, kernel_size=5, stride=2, padding=0),
        nn.BatchNorm2d(16),
        nn.LeakyReLU()
    )
    self.fc1 = nn.Sequential (
        nn.Linear(16*30*30, 128),
        nn.LeakyReLU(),
        nn.Linear(128,1),
        nn.Sigmoid()
    )

  def forward(self, x):
    x = self.main(x)
    x = x.view(-1, 16*30*30)
    x = self.fc1(x)
    return x

#### **Accuracy functions:**

In [ ]:
def compute_G_accuracy(G_model, D_model, dataset, is_use_cuda):
    acc, total = 0, 0
    for i, data in enumerate(dataset, 0):
        original_image, masked_image, _, _ = data

        masked_image = masked_image[0]
        original_image = original_image[0]

        if is_use_cuda and torch.cuda.is_available():
            masked_image = masked_image.cuda()
            original_image = original_image.cuda()

        # get output from generator
        G_output = G_model(masked_image)
        G_output = (G_output * 127.5) + 127.5
        G_output = F.pad(G_output, pad=[52, 52, 52, 52], mode='constant', value=0)

        G_output[:, :, 0:128, 0:52] = original_image[:, :, 0:128, 0:52]
        G_output[:, :, 0:128, 76:128] = original_image[:, :, 0:128, 76:128]
        G_output[:, :, 0:52, 52:76] = original_image[:, :, 0:52, 52:76]
        G_output[:, :, 76:128, 52:76] = original_image[:, :, 76:128, 52:76]

        # get classification result from  discriminator
        D_output = D_model(G_output)
        D_output = D_output.detach().cpu().numpy()

        # calculate acc for image classification
        for i in range(D_output.shape[0]):
            acc += int(D_output[i] > 0.5)
            total += 1

        return acc / total

In [ ]:
def compute_D_accuracy(G_model, D_model, dataset, is_use_cuda):
    acc_real, acc_fake, total_real, total_fake, total_acc = 0, 0, 0 ,0, 0

    # iterate across dataset batches to obtain number of correct classifications
    for i, data in enumerate(dataset, 0):
        original_image, masked_image, _, _ = data 

        masked_image = masked_image[0]
        original_image = original_image[0]

        if is_use_cuda and torch.cuda.is_available():
            masked_image = masked_image.cuda()
            original_image = original_image.cuda()

        # get output from model 
        G_output = G_model(masked_image)
        G_output = (G_output * 127.5) + 127.5
        G_output = F.pad(G_output, pad=[52, 52, 52, 52], mode='constant', value=0)

        G_output[:, :, 0:128, 0:52] = original_image[:, :, 0:128, 0:52]
        G_output[:, :, 0:128, 76:128] = original_image[:, :, 0:128, 76:128]
        G_output[:, :, 0:52, 52:76] = original_image[:, :, 0:52, 52:76]
        G_output[:, :, 76:128, 52:76] = original_image[:, :, 76:128, 52:76]

        # get loss for real images
        output_real = D_model(original_image)
        numpy_output_real = output_real.detach().cpu().numpy()
        # calculate acc for real image classification
        for i in range(numpy_output_real.shape[0]):
            acc_real += int(numpy_output_real[i] > 0.5)
            total_real += 1

        # get loss for fake images
        output_fake = D_model(G_output)
        numpy_output_fake = output_fake.detach().cpu().numpy()
        # calculate acc for fake image classification
        for i in range(numpy_output_fake.shape[0]):
            acc_fake += int(numpy_output_fake[i] < 0.5)
            total_fake += 1

        # calculate total accuracy
        total_acc = (acc_real + acc_fake) / (total_real + total_fake)
        return total_acc              

#### **Loss functions:**

In [ ]:
def compute_G_loss(G_model, D_model, criterion, dataset, is_use_cuda, device):
    i = 0
    total_loss = 0.0
    
    for i, data in enumerate(dataset, 0):
        original_image, masked_image, _, _ = data

        masked_image = masked_image[0]
        original_image = original_image[0]

        if is_use_cuda and torch.cuda.is_available():
            masked_image = masked_image.cuda()
            original_image = original_image.cuda()

        # get output from generator
        G_output = G_model(masked_image)
        G_output = (G_output * 127.5) + 127.5
        G_output = F.pad(G_output, pad=[52, 52, 52, 52], mode='constant', value=0)

        G_output[:, :, 0:128, 0:52] = original_image[:, :, 0:128, 0:52]
        G_output[:, :, 0:128, 76:128] = original_image[:, :, 0:128, 76:128]
        G_output[:, :, 0:52, 52:76] = original_image[:, :, 0:52, 52:76]
        G_output[:, :, 76:128, 52:76] = original_image[:, :, 76:128, 52:76]

        # get output from discriminator
        D_output = D_model(G_output)

        # get loss for each output and the total loss
        loss = criterion(D_output, generate_binary_label(DLabel.REAL, len(D_output), device))
        total_loss += loss.item()
    
    # get average loss from total loss
    total_loss = float(total_loss) / (i+1)
    return total_loss

In [ ]:
def compute_D_loss(G_model, D_model, criterion, dataset, is_use_cuda, device):
    i = 0
    total_loss = 0.0

    for i, data in enumerate(dataset, 0):
        original_image, masked_image, _, _ = data 

        masked_image = masked_image[0]
        original_image = original_image[0]

        if is_use_cuda and torch.cuda.is_available():
            masked_image = masked_image.cuda()
            original_image = original_image.cuda()

        # get output images from generator
        G_output = G_model(masked_image)  
        G_output = (G_output * 127.5) + 127.5
        G_output = F.pad(G_output, pad=[52, 52, 52, 52], mode='constant', value=0)

        G_output[:, :, 0:128, 0:52] = original_image[:, :, 0:128, 0:52]
        G_output[:, :, 0:128, 76:128] = original_image[:, :, 0:128, 76:128]
        G_output[:, :, 0:52, 52:76] = original_image[:, :, 0:52, 52:76]
        G_output[:, :, 76:128, 52:76] = original_image[:, :, 76:128, 52:76]

        # get loss for real images
        output_real = D_model(original_image)
        loss_real = criterion(output_real, generate_binary_label(DLabel.REAL, len(output_real), device))

        # get loss for fake images
        output_fake = D_model(G_output)
        loss_fake = criterion(output_fake, generate_binary_label(DLabel.FAKE, len(output_fake), device))

        # get total loss
        D_loss = loss_real.item() + loss_fake.item()
        total_loss += D_loss
        
    # get average loss from total loss
    total_loss = float(total_loss) / (i+1)
    return total_loss

#### **Helper Functions:**

In [ ]:
def generate_binary_label(dlabel, n, device):
    if dlabel.value == 0:
        return torch.zeros([n, 1], device=device)
    elif dlabel.value == 1:
        return torch.ones([n, 1], device=device)

In [ ]:
def plot_training_curves(train_acc_csv_path=None, train_loss_csv_path=None,
                         valid_acc_csv_path=None, valid_loss_csv_path=None):
    import matplotlib.pyplot as plt

    # plot 1 - Training vs Validation Accuracy
    if train_acc_csv_path or valid_acc_csv_path:
        if train_acc_csv_path:
            train_acc = np.loadtxt(train_acc_csv_path, delimiter=',').transpose()
            plt.plot(train_acc[0], train_acc[1], label="Train")

        if valid_acc_csv_path:
            val_acc = np.loadtxt(valid_acc_csv_path, delimiter=',').transpose()
            plt.plot(val_acc[0], val_acc[1], label="Validation")

        plt.title("Training Curve - Accuracy")
        plt.xlabel("Iterations")
        plt.ylabel("Accuracy")
        plt.legend(loc='best')

        plt.show()

    # plot 2 - Training vs Validation Loss
    if train_loss_csv_path or valid_loss_csv_path:
        if train_loss_csv_path:
            train_loss = np.loadtxt(train_loss_csv_path, delimiter=',').transpose()
            plt.plot(train_loss[0], train_loss[1], label="Train")

        if valid_loss_csv_path:
            val_loss = np.loadtxt(valid_loss_csv_path, delimiter=',').transpose()
            plt.plot(val_loss[0], val_loss[1], label="Validation")

        plt.title("Training Curve - Loss")
        plt.xlabel("Iterations")
        plt.ylabel("Loss")
        plt.legend(loc='best')

        plt.show()

In [ ]:
# custom weights initialization called on netG and netD
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

In [ ]:
# pixel by pixel loss 
def pixel_loss(original_img, generated_img, use_cuda=False):
  """ Returns the loss from pixel by pixel comaparison for one image.
      orginal_img - the img from dataset
      generated_img - the img produced by generator
  """
  if use_cuda and torch.cuda.is_available():
    original_img = original_img.cuda()
    generated_img = generated_img.cuda()

  difference = (original_img - generated_img)**2
  diff = (torch.sum(difference))

  return diff.item()

def batch_pixel_loss(original_imgs, generated_imgs):
  """ Returns the loss from pixel by pixel comaparison for one batch.
      orginal_img - the batch of imgs from dataset
      generated_img - the batch of imgs produced by generator
  """

  loss = 0
  for i in range(original_imgs.shape[0]):
    loss += pixel_loss(original_imgs[i], generated_imgs[i])
  
  return loss/original_imgs.shape[0]


# pixel by pixel accuracy 
def pixel_accuracy(original_img, generated_img, use_cuda=False):
  """ Returns the accuracy from pixel by pixel comaparison for one image.
      orginal_img - the img from dataset
      generated_img - the img produced by generator
  """
  if use_cuda and torch.cuda.is_available():
    original_img = original_img.cuda()
    generated_img = generated_img.cuda()

  difference = original_img - generated_img
  difference = torch.abs(difference)
  # print(original_img)
  # print(generated_img)
  # print(torch.sum(difference))
  diff = 1 - (torch.sum(difference)/(24*24))

  return diff.item()

def batch_pixel_accuracy(original_imgs, generated_imgs):
  """ Returns the accuracy from pixel by pixel comaparison for one batch.
      orginal_img - the batch of imgs from dataset
      generated_img - the batch of imgs produced by generator
  """

  accuracy = 0
  for i in range(original_imgs.shape[0]):
    accuracy += pixel_accuracy(original_imgs[i], generated_imgs[i])
  
  return accuracy/original_imgs.shape[0]

def compute_pixel_loss_accuracy(G_model, dataset, is_use_cuda):
    loss, acc, total = 0, 0, 0
    for i, data in enumerate(dataset, 0):
        original_image, masked_image, _, _ = data

        masked_image = masked_image[0]
        original_image = original_image[0][:, :, 52:76, 52:76]

        if is_use_cuda and torch.cuda.is_available():
            masked_image = masked_image.cuda()
            original_image = original_image.cuda()

        # get output from generator
        G_output = G_model(masked_image)
        G_output = (G_output * 127.5) + 127.5

        loss += batch_pixel_loss(original_image/255, G_output/255)
        acc += batch_pixel_accuracy(original_image/255, G_output/255)
        total += 1

    return (loss/total, acc/total)

#### **Training:**

In [ ]:
def train_dcgan(g_model, d_model,
                train_dataset, valid_dataset, nepochs, id,
                g_criterion=None, d_criterion=None,
                g_niter_freeze=None, d_niter_freeze=None,
                g_nupdate_per_iter=None, d_nupdate_per_iter=None,
                adam_g_lrate=None, adam_g_betas=None, adam_g_eps=None,
                adam_g_weight_decay=None, adam_g_amsgrad=None,
                adam_d_lrate=None, adam_d_betas=None, adam_d_eps=None,
                adam_d_weight_decay=None, adam_d_amsgrad=None,
                nbatches_per_dpt=None, nbatches_per_plot_g_out=None,
                nepochs_per_model_save=None,
                nimgs_per_plot_g_out=None,
                model_dir=None, is_use_cuda=None):
    import datetime
    import os

    device = 'cuda' if is_use_cuda else 'cpu'

    # get batch size
    batch_size = None
    for e in train_dataset:
      batch_size = len(e[0][0])
      break

    # set default args
    if g_criterion is None:
        g_criterion = nn.BCELoss()
    if d_criterion is None:
        d_criterion = nn.BCELoss()

    if g_niter_freeze is None:
        g_niter_freeze = 0
    if d_niter_freeze is None:
        d_niter_freeze = 0

    if g_nupdate_per_iter is None:
        g_nupdate_per_iter = 1
    if d_nupdate_per_iter is None:
        d_nupdate_per_iter = 1

    if adam_g_lrate is None:
        adam_g_lrate = 0.001
    if adam_g_betas is None:
        adam_g_betas = (0.9, 0.999)
    if adam_g_eps is None:
        adam_g_eps = 1e-08
    if adam_g_weight_decay is None:
        adam_g_weight_decay = 0
    if adam_g_amsgrad is None:
        adam_g_amsgrad = False

    if adam_d_lrate is None:
        adam_d_lrate = 0.001
    if adam_d_betas is None:
        adam_d_betas = (0.9, 0.999)
    if adam_d_eps is None:
        adam_d_eps = 1e-08
    if adam_d_weight_decay is None:
        adam_d_weight_decay = 0
    if adam_d_amsgrad is None:
        adam_d_amsgrad = False

    if nbatches_per_plot_g_out is None:
        nbatches_per_plot_g_out = 25
    if nbatches_per_dpt is None:
        nbatches_per_dpt = 5
    if nepochs_per_model_save is None:
        nepochs_per_model_save = 50
    if nimgs_per_plot_g_out is None:
        nimgs_per_plot_g_out = min(10, batch_size)
    else:
        nimgs_per_plot_g_out = min(nimgs_per_plot_g_out, batch_size)
    if model_dir is None:
        model_dir = "trained_models"
    if is_use_cuda is None:
        is_use_cuda = False

    # keep track of time elapsed
    start_time = time.time()

    # create model directory if does not exist
    os.makedirs(model_dir, exist_ok=True)

    # store hyperparameter values to txt file
    with open(rf"{model_dir}\{id}_hyperparameter_specs.txt", 'w') as file:
        file.write(f"timestamp: {datetime.datetime.now()}")
        file.write(f"\n")
        file.write(f"model id: {id}\n")
        file.write(f"batch size: {batch_size}\n")
        file.write(f"number of batches: {len(train_dataset)}\n")
        file.write(f"number of epochs: {nepochs}\n")
        file.write(f"number of batches per datapoint: {nbatches_per_dpt}\n")
        file.write(f"number of batches per plot generator output: {nbatches_per_plot_g_out}\n")
        file.write(f"number of epochs per model save: {nepochs_per_model_save}\n")
        file.write(f"number of images per plot generator output: {nimgs_per_plot_g_out}\n")
        file.write(f"\n")
        file.write(f"GENERATOR\n")
        file.write(f"\t number of iterations to freeze: {g_niter_freeze}\n")
        file.write(f"\t number of updates per iteration: {g_nupdate_per_iter}\n")
        file.write(f"\t learning rate: {adam_g_lrate}\n")
        file.write(f"\t betas: {adam_g_betas}\n")
        file.write(f"\t eps: {adam_g_eps}\n")
        file.write(f"\t weight decay: {adam_g_weight_decay}\n")
        file.write(f"\t amsgrad: {adam_g_amsgrad}\n")
        file.write(f"\n")
        file.write(f"GLOBAL DISCRIMINATOR\n")
        file.write(f"\t number of iterations to freeze: {d_niter_freeze}\n")
        file.write(f"\t number of updates per iteration: {d_nupdate_per_iter}\n")
        file.write(f"\t learning rate: {adam_d_lrate}\n")
        file.write(f"\t betas: {adam_d_betas}\n")
        file.write(f"\t eps: {adam_d_eps}\n")
        file.write(f"\t weight decay: {adam_d_weight_decay}\n")
        file.write(f"\t amsgrad: {adam_d_amsgrad}\n")

    # set Adam optimizer for each model
    g_optimizer = torch.optim.Adam(g_model.parameters(),
                                   lr=adam_g_lrate, betas=adam_g_betas,
                                   eps=adam_g_eps, weight_decay=adam_g_weight_decay,
                                   amsgrad=adam_g_amsgrad)
    d_optimizer = torch.optim.Adam(d_model.parameters(),
                                   lr=adam_d_lrate, betas=adam_d_betas,
                                   eps=adam_d_eps, weight_decay=adam_d_weight_decay,
                                   amsgrad=adam_d_amsgrad)

    # train model
    train_acc_g = list()
    train_loss_g = list()
    val_acc_g = list()
    val_loss_g = list()
    train_acc_d = list()
    train_loss_d = list()
    val_acc_d = list()
    val_loss_d = list()
    train_pixel_acc_g = list()
    train_pixel_loss_g = list()
    val_pixel_acc_g = list()
    val_pixel_loss_g = list()
    niter = nepochs * len(train_dataset)
    iter = 0
    for epoch in range(1, nepochs + 1):
        # iterate through each batch in training dataset
        for ori_imgs, pre_proc_imgs, _, _ in train_dataset:
            iter += 1

            ori_imgs = ori_imgs[0]
            pre_proc_imgs = pre_proc_imgs[0]

            # store in GPU for CUDA processing
            if is_use_cuda and torch.cuda.is_available():
                ori_imgs = ori_imgs.cuda()
                pre_proc_imgs = pre_proc_imgs.cuda()

            # A) train global discriminator
            #d_loss = None
            if (iter - 1) % (d_niter_freeze + 1) == 0:
                for _ in range(d_nupdate_per_iter):
                    g_output = g_model(pre_proc_imgs)
                    g_output = (g_output * 127.5) + 127.5
                    g_output = F.pad(g_output, pad=[52, 52, 52, 52], mode='constant', value=0)

                    g_output[:, :, 0:128, 0:52] = ori_imgs[:, :, 0:128, 0:52]
                    g_output[:, :, 0:128, 76:128] = ori_imgs[:, :, 0:128, 76:128]
                    g_output[:, :, 0:52, 52:76] = ori_imgs[:, :, 0:52, 52:76]
                    g_output[:, :, 76:128, 52:76] = ori_imgs[:, :, 76:128, 52:76]
                    #cropped = torch.round(cropped)

                    d_model.zero_grad()
                    # A.a) real - data from training set
                    d_output_real = d_model(ori_imgs)
                    d_loss_real = d_criterion(d_output_real,
                                              generate_binary_label(DLabel.REAL,
                                                                    len(d_output_real),
                                                                    device))
                    d_loss_real.backward()

                    # A.b) fake - data outputted by generator model
                    d1_output_fake = d_model(g_output)
                    d1_loss_fake = d_criterion(d1_output_fake,
                                               generate_binary_label(DLabel.FAKE,
                                                                     len(d1_output_fake),
                                                                     device))

                    d1_loss_fake.backward()

                    d_loss = d_loss_real + d1_loss_fake
                    d_optimizer.step()

            # B) train generator
            # g_loss = None
            # g_output = None
            if (iter - 1) % (g_niter_freeze + 1) == 0:
                for _ in range(g_nupdate_per_iter):
                    g_model.zero_grad()
                    g_output = g_model(pre_proc_imgs)
                    g_output = (g_output * 127.5) + 127.5
                    g_output = F.pad(g_output, pad=[52, 52, 52, 52], mode='constant', value=0)

                    g_output[:, :, 0:128, 0:52] = ori_imgs[:, :, 0:128, 0:52]
                    g_output[:, :, 0:128, 76:128] = ori_imgs[:, :, 0:128, 76:128]
                    g_output[:, :, 0:52, 52:76] = ori_imgs[:, :, 0:52, 52:76]
                    g_output[:, :, 76:128, 52:76] = ori_imgs[:, :, 76:128, 52:76]

                    d_output_fake_2 = d_model(g_output)

                    g_loss = g_criterion(d_output_fake_2,
                                         generate_binary_label(DLabel.REAL,
                                                               len(d_output_fake_2),
                                                               device))
                    g_loss.backward()
                    g_optimizer.step()

            # store datapoint w/ frequency based on 'nbatches_per_dpt' and at last iteration
            if (iter - 1) % nbatches_per_dpt == 0 or iter == niter:
                # compute and record accuracy / loss
                # a) generator
                train_acc_g.append((iter, compute_G_accuracy(g_model, d_model,
                                                             train_dataset,
                                                             is_use_cuda=is_use_cuda)))
                train_loss_g.append((iter, g_loss.item()))
                val_acc_g.append((iter, compute_G_accuracy(g_model, d_model,
                                                           valid_dataset,
                                                           is_use_cuda=is_use_cuda)))
                val_loss_g.append((iter, compute_G_loss(g_model, d_model,
                                                        g_criterion,
                                                        valid_dataset,
                                                        is_use_cuda=is_use_cuda, device=device)))             

                # b) discriminator
                train_acc_d.append((iter, compute_D_accuracy(g_model, d_model,
                                                             train_dataset,
                                                             is_use_cuda=is_use_cuda)))
                train_loss_d.append((iter, d_loss.item()))
                val_acc_d.append((iter, compute_D_accuracy(g_model, d_model,
                                                           valid_dataset,
                                                           is_use_cuda=is_use_cuda)))
                val_loss_d.append((iter, compute_D_loss(g_model, d_model,
                                                        d_criterion,
                                                        valid_dataset,
                                                        is_use_cuda=is_use_cuda, device=device)))
                
                # c) pixel loss
                px_train_loss, px_train_acc = compute_pixel_loss_accuracy(g_model,
                                                                          train_dataset,
                                                                          is_use_cuda=is_use_cuda)
                px_val_loss, px_val_acc = compute_pixel_loss_accuracy(g_model,
                                                                      valid_dataset,
                                                                      is_use_cuda=is_use_cuda)
                train_pixel_acc_g.append((iter, px_train_acc))
                train_pixel_loss_g.append((iter, px_train_loss))
                val_pixel_acc_g.append((iter, px_val_acc))
                val_pixel_loss_g.append((iter, px_val_loss))

                # output accuracy / loss
                print(f"Epoch {epoch} | iter {iter}: ")
                # a) generator
                print("a) generator")
                print(f"\t training accuracy: {train_acc_g[-1][1]}")
                print(f"\t training loss: {train_loss_g[-1][1]}")
                print(f"\t validation accuracy: {val_acc_g[-1][1]}")
                print(f"\t validation loss: {val_loss_g[-1][1]}")
                # b) discriminator
                print("b) discriminator")
                print(f"\t training accuracy: {train_acc_d[-1][1]}")
                print(f"\t training loss: {train_loss_d[-1][1]}")
                print(f"\t validation accuracy: {val_acc_d[-1][1]}")
                print(f"\t validation loss: {val_loss_d[-1][1]}")
                # c) pixel loss
                print("c) pixel loss")
                print(f"\t training accuracy: {train_pixel_acc_g[-1][1]}")
                print(f"\t training loss: {train_pixel_loss_g[-1][1]}")
                print(f"\t validation accuracy: {val_pixel_acc_g[-1][1]}")
                print(f"\t validation loss: {val_pixel_loss_g[-1][1]}")

            # plot output w/ frequency based on 'nbatches_per_plot_g_out' and at last iteration
            if (iter - 1) % nbatches_per_plot_g_out == 0 or iter == niter:
                print(f"Epoch {epoch} | iter {iter}: "
                      f"generated img - its missing region - original img - its missing region")
                fig = plt.figure(figsize=(15, 15))
                cols = 4
                rows = nimgs_per_plot_g_out

                for r in range(1, rows + 1):
                    i = (r - 1) * cols

                    # a) plot generated image
                    fig.add_subplot(rows, cols, i + 1)
                    plt.imshow(np.transpose(g_output[r-1].detach().cpu() / 255,
                                            (1, 2, 0)))

                    # b) plot missing region of generated image
                    fig.add_subplot(rows, cols, i + 2)
                    plt.imshow(np.transpose(g_output[r-1][:, 52:76, 52:76].detach().cpu() / 255,
                                            (1, 2, 0)))

                    # c) plot original image
                    fig.add_subplot(rows, cols, i + 3)
                    plt.imshow(np.transpose(ori_imgs[r-1].detach().cpu() / 255,
                                            (1, 2, 0)))

                    # d) plot missing region of original image
                    fig.add_subplot(rows, cols, i + 4)
                    plt.imshow(np.transpose(ori_imgs[r-1][:, 52:76, 52:76].detach().cpu() / 255,
                                            (1, 2, 0)))
                plt.show()

        # save frequency of the current model (checkpoint) to a file based on
        # 'nepochs_per_model_save' and at last epoch
        if (epoch - 1) % nepochs_per_model_save == 0 or epoch == nepochs:
            # a) generator
            path = rf"{model_dir}\{id}_G_ne{nepochs}_niter{niter}_e{epoch}_iter{iter}"
            torch.save(g_model.state_dict(), path)
            # b) global discriminator
            path = rf"{model_dir}\{id}_D1_ne{nepochs}_niter{niter}_e{epoch}_iter{iter}"
            torch.save(d_model.state_dict(), path)

    # output elapsed time
    print('finished training')
    end_time = time.time()
    elapsed_time = end_time - start_time
    print("total time elapsed: {:.2f} seconds".format(elapsed_time))

    # write the train/test loss/err into CSV file for plotting later
    # a) generator
    g_csv_paths = (rf"{model_dir}\{id}_G_train_accuracy.csv",
                   rf"{model_dir}\{id}_G_train_loss.csv",
                   rf"{model_dir}\{id}_G_val_accuracy.csv",
                   rf"{model_dir}\{id}_G_val_loss.csv")
    np.savetxt(g_csv_paths[0], train_acc_g, delimiter=',')
    np.savetxt(g_csv_paths[1], train_loss_g, delimiter=',')
    np.savetxt(g_csv_paths[2], val_acc_g, delimiter=',')
    np.savetxt(g_csv_paths[3], val_loss_g, delimiter=',')
    # b) discriminator
    d1_csv_paths = (rf"{model_dir}\{id}_D_train_accuracy.csv",
                    rf"{model_dir}\{id}_D_train_loss.csv",
                    rf"{model_dir}\{id}_D_val_accuracy.csv",
                    rf"{model_dir}\{id}_D_val_loss.csv")
    np.savetxt(d1_csv_paths[0], train_acc_d, delimiter=',')
    np.savetxt(d1_csv_paths[1], train_loss_d, delimiter=',')
    np.savetxt(d1_csv_paths[2], val_acc_d, delimiter=',')
    np.savetxt(d1_csv_paths[3], val_loss_d, delimiter=',')
    # c) pixel loss
    px_csv_paths = (rf"{model_dir}\{id}_PX_train_accuracy.csv",
                    rf"{model_dir}\{id}_PX_train_loss.csv",
                    rf"{model_dir}\{id}_PX_val_accuracy.csv",
                    rf"{model_dir}\{id}_PX_val_loss.csv")
    np.savetxt(px_csv_paths[0], train_pixel_acc_g, delimiter=',')
    np.savetxt(px_csv_paths[1], train_pixel_loss_g, delimiter=',')
    np.savetxt(px_csv_paths[2], val_pixel_acc_g, delimiter=',')
    np.savetxt(px_csv_paths[3], val_pixel_loss_g, delimiter=',')

    return g_csv_paths, d1_csv_paths, px_csv_paths

#### **Tunning:**

In [ ]:
# A. create models
# g_model = Generator()
g_model = Generator()
# g_model = Generator_AD()
# d_model = Discriminator()
d_model = Discriminator()
# d_model = Discriminator_3L()
# d_model = Discriminator_3LC()
# d_model = Discriminator_4LC_avgP()
# d_model = Discriminator_3LC_avgP()

# weight initialization
g_model.apply(weights_init)
d_model.apply(weights_init)

is_use_cuda = True
if is_use_cuda and torch.cuda.is_available():
    g_model.cuda()
    d_model.cuda()

# B. create dataset
bs = 64
t_ind = 0.75
v_ind = 0.15
o_num = 1

t_loader, v_loader, test_loader, overfit_loader = data_loader(bs=bs,
                                                              t_ind=t_ind,
                                                              v_ind=v_ind,
                                                              o_num=o_num,
                                                              dataset=dataset_combined)

# C. train models
train_dataset = t_loader
valid_dataset = v_loader
overfit_dataset = overfit_loader

nepochs = 1500
# id = "test"
id = "bldg_rgb-gED_d2L-ne1000"

g_niter_freeze = 0
d_niter_freeze = 0
g_nupdate_per_iter = 1
d_nupdate_per_iter = 2

adam_g_lrate = 0.00078
adam_g_betas = None
adam_g_eps = None
adam_g_weight_decay = None
adam_g_amsgrad = None

adam_d_lrate = 0.00078
adam_d_betas = None
adam_d_eps = None
adam_d_weight_decay = None
adam_d_amsgrad = None

nbatches_per_dpt = 1000
nbatches_per_plot_g_out = 1000
nepochs_per_model_save = 1000
nimgs_per_plot_g_out = 1

csv_paths = train_dcgan(g_model, d_model,
                        train_dataset, valid_dataset, nepochs, id,
                        g_niter_freeze=g_niter_freeze, d_niter_freeze=d_niter_freeze,
                        g_nupdate_per_iter=g_nupdate_per_iter,
                        d_nupdate_per_iter=d_nupdate_per_iter,
                        adam_g_lrate=adam_g_lrate, adam_g_betas=adam_g_betas, adam_g_eps=adam_g_eps,
                        adam_g_weight_decay=adam_g_weight_decay, adam_g_amsgrad=adam_g_amsgrad,
                        adam_d_lrate=adam_d_lrate, adam_d_betas=adam_d_betas, adam_d_eps=adam_d_eps,
                        adam_d_weight_decay=adam_d_weight_decay, adam_d_amsgrad=adam_d_amsgrad,
                        nbatches_per_dpt=nbatches_per_dpt,
                        nbatches_per_plot_g_out=nbatches_per_plot_g_out,
                        nepochs_per_model_save=nepochs_per_model_save,
                        nimgs_per_plot_g_out=nimgs_per_plot_g_out,
                        is_use_cuda=is_use_cuda)
g_csv_paths, d_csv_paths, px_csv_path = csv_paths

# D. plot
print("training curves - generator")
plot_training_curves(*g_csv_paths)
print('')
print("training curves - discriminator")
plot_training_curves(*d_csv_paths)
print('')
print("training curves - pixel difference")
plot_training_curves(*px_csv_path)